# Функция как объект

Функция в R -- объект "первого класса", её можно:

. . .

Использовать как обычный объект

In [2]:
str( c(mean, max) )
fun_list <- c( mean, max )
sapply(fun_list, function(f) f(1:100))

List of 2
 $ :function (x, ...)  
 $ :function (..., na.rm = FALSE)  


[1]  50.5 100.0

# Функция как аргумент

. . .

Указывать в качестве аргумента


In [8]:
apply_f <- function(f, x) f(x)
sapply(fun_list, apply_f, x = 1:100)

[1]  50.5 100.0

... при этом анонимная функция тоже подойдёт

In [10]:
apply_f(function(x) sum(x^2), 1:10)

[1] 385

# Функция как return value

Использовать как возвращаемое значение


In [85]:
square <- function() function(x) x^2
square()
square()(5)

function(x) x^2
<environment: 0x00000000159cfac8>

[1] 25

# Функции внутри функций

. . .

In [4]:
f <- function(x) {
  g <- function(y) if (y > 0) 1 else if (y < 0) -1 else 0
  sapply(x, g)
}
all.equal(f(-100:100), sign(-100:100))

[1] TRUE

. . .

Идеальный случай: если функция `g` нужна только внутри функции `f` и не очень громоздка

Функция внутри функции -- один из вариантов инкапсуляции в R

# Исходный код функции


. . .

Простейший случай: напечатать имя функции без скобок (напр., `sd`)

In [5]:
(f <- function(x) x^5)

function(x) x^5

. . .

Если в выводе есть `.C`, `.Call`, `.Fortran`, `.External`, `.Internal`, `.Primitive`, то это обращение к скомпилированному коду: нужно смотреть исходный код R (напр., `var`)

. . .

Если в выводе есть `UseMethod` или `standardGeneric`, то это method dispatch для классов S3/S4 (полиморфизм; напр., `plot`)


In [15]:
methods(plot)[1:20]

[1] "plot.acf"           "plot.data.frame"    "plot.decomposed.ts"
 [4] "plot.default"       "plot.dendrogram"    "plot.density"      
 [7] "plot.ecdf"          "plot.factor"        "plot.formula"      
[10] "plot.function"      "plot.hclust"        "plot.histogram"    
[13] "plot.HoltWinters"   "plot.isoreg"        "plot.lm"           
[16] "plot.medpolish"     "plot.mlm"           "plot.ppr"          
[19] "plot.prcomp"        "plot.princomp"

Полное описание всех случаев: [http://stackoverflow.com/questions/19226816/how-can-i-view-the-source-code-for-a-function](http://stackoverflow.com/questions/19226816/how-can-i-view-the-source-code-for-a-function)

# Возвращаемое значение

. . .

Определяется либо ключевым словом `return`: 

In [17]:
has_na <- function(v) {
  for (k in v) if (is.na(k)) return(TRUE)
  return(FALSE)
}

. . .

либо последним вычисленным значением:

In [18]:
has_na <- function(v) any(is.na(v))

# Аргументы по умолчанию

. . .

Посмотрите, как объявлена функция `seq`:


In [20]:
seq(from = 1, to = 1, by = ((to - from)/(length.out - 1)),
    length.out = NULL, along.with = NULL)

integer(0)

>- Аргументы могут иметь значения по умолчанию
>- Значения могут вычисляться на лету!

In [21]:
seq() # from = 1, to = 1
seq(1, 5, length.out = 11) # by = (5 - 1)/(11 - 1)

[1] 1

[1] 1.0 1.4 1.8 2.2 2.6 3.0 3.4 3.8 4.2 4.6 5.0

# Правила разбора аргументов

. . .

Рассмотрим на примере:


In [18]:
f <- function(arg1, arg2, remove_na = TRUE, ..., optional_arg) {}
f(1, arg2 = 2, remove = F, optional_arg = 42, do_magic = TRUE)

NULL

Разбор аргументов проходит в три этапа:

1. Точное совпадение имени аргумента -- `arg2`, `optional_arg`
2. Частичное совпадение имени аргумента (только до `...`) -- `remove_na`
3. Разбор аргументов по позиции -- `arg1`

Неразобранные аргументы попадают в `...` -- `do_magic`

# Проброс аргументов

Один случай использования ellipsis -- "произвольное количество передаваемых объектов", функции `sum`, `c`, `cbind`, `paste`

. . .

Другой характерный случай -- "проброс аргументов":


In [17]:
f <- function(x, pow = 2) x^pow
integrate(f, 0, 1) # lower = 0, upper = 1, pow = 2
integrate(f, 0, 1, pow = 5) # same, but pow = 5

0.3333333 with absolute error < 3.7e-15

0.1666667 with absolute error < 1.9e-15

# Бинарные операторы

Оператор `x %in% y`: есть ли вхождения элементов `x` в `y`? 

In [24]:
1:5 %in% c(1, 2, 5)

[1]  TRUE  TRUE FALSE FALSE  TRUE

In [26]:
"%nin%" <- function(x, y) !(x %in% y)
1:5 %nin% c(1, 2, 5)

[1] FALSE FALSE  TRUE  TRUE FALSE

# Глоссарий

. . .

`?"function"`

Source code for functions, `?methods`

Argument matching, ellipsis (`?"..."`)

Чтобы глубже прочувствовать частичное дополнение аргументов функции, давайте рассмотрим "полный" вызов функции seq (x, y и z -- числа): 
```r
seq(from = x, to = y, by = z) 
```
Какие из следующих вызовов эквивалентны этому? 
Убедитесь, что вы понимаете, почему так происходит.
P.S. Злоупотреблять этими возможностями не стоит. Выбирайте разумный компромисс между частичным дополнением и удобочитаемостью.


In [9]:
x = 1
y = 5
z = 2

In [11]:
seq(from =x,to= y, by = z)

[1] 1 3 5

In [12]:
seq(y, z, fr = x)

[1] 1 3 5

In [13]:
seq(b = z, f = x, t = y)

[1] 1 3 5

In [14]:
seq(to = y, by = z, from = x)

[1] 1 3 5

In [15]:
seq(by = z, x,y)

[1] 1 3 5

Одна интересная конструкция, о которой я рассказал, может быть использована для передачи произвольного количества аргументов далее по стеку вызовов вложенных функций. С помощью этой конструкции напишите такую функцию, которая займётся украшением строк.

Пусть функция decorate_string действует поверх функции paste, дополнительно приклеивая к результату аргумент pattern. При этом этот аргумент должен быть присоединён как в начале строки (строк), так и в конце, но перевёрнутый задом наперёд.

Тут проще всего показать на примерах:
```r
decorate_string(pattern = "123", "abc")            # "123abc321"
decorate_string(pattern = "123", "abc", "def")     # "123abc def321"
decorate_string(pattern = "123", c("abc", "def"))  # "123abc321" "123def321" (вектор длины 2)
Обратите внимание, что функция decorate_string должна помнить про аргументы для paste и правильно на них реагировать:

decorate_string(pattern = "123", "abc", "def", sep = "+")    # "123abc+def321"
decorate_string(pattern = "!", c("x", "x"), collapse = "_")  # "!x_x!"
decorate_string(pattern = ".:", 1:2, 3:4, 5:6, sep = "&")    # ".:1&3&5:." ".:2&4&6:." (вектор длины 2)

```
Подсказки: 
если вам нужна утилитарная функция для какого-то простого промежуточного действия, напишите и проверьте её отдельно;
в шаблоне для отправки <???> означает то место, которое нужно написать самостоятельно;
убедитесь, что результат для вызовов выше в точности такой, как я указал!
P.S. Функции такого рода не праздное развлечение: похожие задачи могут возникать при построении путей к файлам или, например, HTTP запросов.

In [185]:
decorate_string <- function(pattern,..., se, colla ) { 
  d <- paste(...)
  p2 <- paste(rev(  rapply(strsplit(pattern, ""), c)) , collapse = ""  )
  print(paste(pattern,d,p2, sep = ""))
}

In [186]:
decorate_string(pattern = "123", "abc")                                  # "123abc321"

[1] "123abc321"


In [187]:
decorate_string(pattern = "123", "abc", "def")                           # "123abc def321"

[1] "123abc def321"


In [188]:
decorate_string(pattern = "123", c("abc", "def"))                        # "123abc321" "123def321" (вектор длины 2)

[1] "123abc321" "123def321"


In [189]:
decorate_string(pattern = "123", "abc", "def", sep = "+")                # "123abc+def321"

[1] "123abc+def321"


In [190]:
decorate_string(pattern = "!", c("x", "x"), collapse = "_")              # "!x_x!"

[1] "!x_x!"


In [191]:
decorate_string(pattern = ".:", 1:2, 3:4, 5:6, sep = "&")                # ".:1&3&5:." ".:2&4&6:." (вектор длины 2)

[1] ".:1&3&5:." ".:2&4&6:."


In [195]:
paste("number", LETTERS[1:3], 1:6, sep = "_", collapse = " and ")

[1] "number_A_1 and number_B_2 and number_C_3 and number_A_4 and number_B_5 and number_C_6"

In [197]:
lapply(1:4, rnorm, mean = 10, sd = 10)

[[1]]
[1] 19.48936

[[2]]
[1] -8.542783  9.232832

[[3]]
[1] 12.011485  6.977169 13.498414

[[4]]
[1] -9.856314  8.525674 20.091875 18.935208

In [198]:
sort(rnorm(5))

[1] -2.05966427 -0.83650175  0.05456786  0.58600379  0.69066398

In [199]:
matrix(nrow = 3, ncol = 2, by = T, 0, dimn = NULL)

0,0
0,0
0,0


К этому времени у нас уже есть полный набор для приятного времяпрепровождения: монетка, игральный кубик и даже колода карт. Давайте откроем казино!

Для создания респектабельного заведения нам не хватает игры в рулетку. Это единственное, что отделяет нас от заветного "Делайте ваши ставки, дамы и господа!"

И, раз уж вы займётесь изготовлением рулетки, у меня к вам одна деликатная просьба: не могли бы вы устроить так, чтобы их было две? При этом одна будет абсолютно нормальной, а вторая, ну, как бы так выразиться... Ведь если зеро будет выпадать чуточку почаще, то казино получит больше денег...

Воспользуйтесь подготовленным мной шаблоном, чтобы получить две функции, содержащие честную и нечестную рулетку.

Честная -- это когда все имеющиеся значения (всего их 37) выпадают с равной вероятностью.

А нечестная пусть выдаёт все значения, кроме зеро, с равной вероятностью. Что же касается зеро (первый элемент определённого мной вектора roulette_values), то вероятность его выпадения пусть будет в два раза больше, чем любого другого значения. Не переборщите, иначе игроки заподозрят неладное!

Как и в предыдущем видео, функция generator -- фабрика функций, но с ещё более широкими возможностями. Обратите внимание, как я объявил аргумент prob по умолчанию.

Я ожидаю, что в этом случае вызов generator(roulette_values) без аргумента prob даст функцию для вызова честной рулетки fair_roulette. Для нечестной рулетки rigged_roulette вызов уточните самостоятельно.

Я буду проверять рулетки вызовом обеих функций, так что, пожалуйста, убедитесь, что это функции, для которых обращение вида fair_roulette(n) и  rigged_roulette(n) приведёт к получению n независимых бросков.

In [44]:
# barplot(table(sample(c("Zero!", 1:36), size=100, replace=TRUE, prob=c(0.5,rep(0.5/36, 36))   )))

In [67]:
generator <- function(set, prob = rep(1/length(set), length(set)),n) { 
    return( sample(roulette_values,size =  n, p = prob,replace = TRUE) )
} 
roulette_values <- c("Zero!", 1:36)
fair_roulette <- function(p) generator(roulette_values, n = p)
rigged_roulette <- function(p) generator(  roulette_values, prob = c(1,rep(0.5,36)) , n = p)

In [68]:
fair_roulette(100)

[1] "19"    "7"     "31"    "23"    "12"    "36"    "15"    "35"    "2"    
 [10] "4"     "7"     "15"    "17"    "8"     "27"    "31"    "29"    "13"   
 [19] "2"     "24"    "10"    "17"    "25"    "34"    "10"    "9"     "16"   
 [28] "5"     "1"     "20"    "26"    "24"    "3"     "20"    "17"    "27"   
 [37] "12"    "Zero!" "6"     "17"    "9"     "18"    "28"    "3"     "31"   
 [46] "15"    "3"     "3"     "16"    "33"    "13"    "34"    "2"     "29"   
 [55] "35"    "28"    "16"    "36"    "2"     "13"    "21"    "16"    "18"   
 [64] "4"     "24"    "21"    "13"    "17"    "29"    "1"     "25"    "20"   
 [73] "16"    "33"    "9"     "4"     "9"     "28"    "22"    "16"    "26"   
 [82] "27"    "21"    "17"    "8"     "36"    "4"     "30"    "23"    "9"    
 [91] "2"     "14"    "17"    "19"    "17"    "26"    "25"    "32"    "Zero!"
[100] "17"

In [69]:
rigged_roulette(100)

[1] "24"    "27"    "4"     "20"    "2"     "14"    "32"    "25"    "16"   
 [10] "32"    "11"    "35"    "2"     "31"    "33"    "9"     "6"     "16"   
 [19] "4"     "31"    "34"    "21"    "4"     "26"    "Zero!" "14"    "21"   
 [28] "1"     "7"     "8"     "29"    "6"     "7"     "21"    "34"    "2"    
 [37] "18"    "32"    "20"    "14"    "3"     "25"    "Zero!" "23"    "Zero!"
 [46] "6"     "32"    "28"    "9"     "26"    "32"    "34"    "23"    "16"   
 [55] "1"     "3"     "32"    "34"    "5"     "27"    "Zero!" "14"    "24"   
 [64] "33"    "15"    "29"    "30"    "24"    "11"    "21"    "Zero!" "8"    
 [73] "9"     "31"    "24"    "26"    "4"     "16"    "23"    "Zero!" "35"   
 [82] "Zero!" "2"     "4"     "19"    "26"    "32"    "35"    "12"    "26"   
 [91] "30"    "24"    "25"    "2"     "21"    "36"    "33"    "10"    "18"   
[100] "31"

In [76]:
norm  #svd

function (x, type = c("O", "I", "F", "M", "2")) 
{
    if (identical("2", type)) {
        svd(x, nu = 0L, nv = 0L)$d[1L]
    }
    else .Internal(La_dlange(x, type))
}
<bytecode: 0x000000002091ebf8>
<environment: namespace:base>

Давайте напишем бинарный оператор! Пусть %+% действует на два числовых вектора, складывая их поэлементно, но без учёта правил переписывания: если длина векторов различна, то возвращаем вектор большей длины, но с пропущенными значениями в конце.

Например, 
```r
1:5 %+% 1:2   # c(2, 4, NA, NA, NA)
5 %+% c(2, 6) # c(7, NA) 
```

In [72]:
"%+%" <- function(x,y){
    a = abs(length(x) - length(y))
    p <- c()
    if(a==0){
        return (x+y)
    }else if(length(x)> length(y)){
        for (i in c(1:length(y))){
            p <- append(p, x[i] + y[i])
        }
        return(append(p, rep(NA, a)))
    }else if(length(x)< length(y)){
        for (i in c(1:length(x))){
            p <- append(p, x[i] + y[i])
        }
        return(append(p, rep(NA, a)))
    }
}

In [73]:
1:5 %+% 1:2 

[1]  2  4 NA NA NA

In [74]:
5 %+% c(2, 6)

[1]  7 NA

In [50]:
length(1:2) - length(1:5)

[1] -3

In [71]:
print(append(c(2,4), rep(NA, 5)))

[1]  2  4 NA NA NA NA NA


In [1]:
4%%6

[1] 4

In [5]:
5%%3

[1] 2

In [7]:
max(1:10)
which.max(4:7)
max(c("AA", "BB", "C"))
max(c("50", "2", "C"))
which.max(c("AA", "BB", "C"))
which.max(c("1", "3", "C"))
which(1)

[1] 10

[1] 4

[1] "C"

[1] "C"

Warning message in which.max(c("AA", "BB", "C")):
"в результате преобразования созданы NA"

integer(0)

Warning message in which.max(c("1", "3", "C")):
"в результате преобразования созданы NA"

[1] 2

ERROR: Error in which(1): аргумент 'which' -- не логический


In [8]:
which.max(TRUE,FALSE)

ERROR: Error in which.max(TRUE, FALSE): неиспользованный аргумент (FALSE)


In [9]:
max(TRUE,FALSE)

[1] 1

In [14]:
which.max('a','6')

ERROR: Error in which.max("a", "6"): неиспользованный аргумент ("6")
